<a href="https://colab.research.google.com/github/Guo-Weiqiang/Master-Project/blob/main/SEED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import matplotlib.pyplot as plt

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class SeedDataset(Dataset):
    def __init__(self, file_path):
        self.data = np.load(file_path)
        # self.labels = [i + 1 for i in [1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1] ] * 3
        self.labels = [2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 1, 2, 0] * 3 #
        assert self.data.shape[1:] == (1, 62, 37001), "Invalid shape of data"

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load the .npy file at the specified index
        sample = self.data[idx]

        # Convert the data to a PyTorch tensor
        sample = torch.from_numpy(sample).float()  # Ensure data is float

        # Return the data and a dummy label or the actual label if you have it
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)

        return sample, label

In [4]:
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")

In [2]:
# class EEGNet_ReLU(torch.nn.Module):
#     def __init__(self, n_output):
#         super(EEGNet_ReLU, self).__init__()
#         self.firstConv = nn.Sequential(
#             nn.Conv2d(1, 16, kernel_size=(1,51), stride=(1,1), padding=(0,25),bias=False),
#             nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#         )
#         self.depthwiseConv = nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=(2,1), stride=(1,1), groups=8,bias=False),
#             nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#             nn.ReLU(),
#             nn.AvgPool2d(kernel_size=(1,4), stride=(1,4),padding=0),
#             nn.Dropout(p=0.35)
#         )
#         self.separableConv = nn.Sequential(
#             nn.Conv2d(32, 32, kernel_size=(1,15), stride=(1,1), padding=(0,7),bias=False),
#             nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#             nn.ReLU(),
#             nn.AvgPool2d(kernel_size=(1,8), stride=(1,8),padding=0),
#             nn.Dropout(p=0.35),
#             nn.Flatten()
#         )

#         self.classify = nn.Sequential(
#             nn.Linear(2256512, 256, bias=True),
#             nn.ReLU(),
#             nn.Linear(256, n_output, bias=True)
#         )

#     def forward(self, x):
#         out = self.firstConv(x)
#         out = self.depthwiseConv(out)
#         features = self.separableConv(out)
#         # print('the shape of features before the classifier is ', features.shape)
#         out = self.classify(features)
#         return out, features
class EEGNet_ReLU(torch.nn.Module):
    def __init__(self, n_output):
        super(EEGNet_ReLU, self).__init__()
        # Adjusted kernel sizes and added more pooling
        self.firstConv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(1, 4))
        )
        # Since we have only one channel, the depthwise convolution is just a regular convolution here
        # Adjust the number of output channels if necessary
        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(62, 1), padding=0, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(1, 8)),
            nn.Flatten()
        )
        # The separable convolution may not be needed, depending on the dimensionality after depthwiseConv
        self.classify = nn.Sequential(
            # The input features to the classifier need to be calculated based on the previous layer's output

            nn.Linear(36992, 256, bias=True),
            nn.ReLU(),
            nn.Linear(256, n_output, bias=True)
        )

    def forward(self, x):
        out = self.firstConv(x)
        features = self.depthwiseConv(out)
        # print(features.shape)
        # out will have reduced dimensionality, you need to calculate 'in_features' based on this.
        out = self.classify(features)
        return out, features



def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        print("1", X.shape, y.shape)
        X, y = X.to(DEVICE), y.to(DEVICE)

        # Compute prediction and loss
        pred, features = model(X)
        loss = loss_fn(pred, y)
        correct = 0
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            correct /= size
            print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred, features = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

def main():
    model = EEGNet_ReLU(3)
    model.to(DEVICE)
    summary(model,(1, 62, 37001))

    train_data = SeedDataset('drive/MyDrive/EEGNet/processed_seed_dataset/subject1.npy')
    test_data = SeedDataset('drive/MyDrive/EEGNet/processed_seed_dataset/subject2.npy')

    train_dataloader = DataLoader(train_data, batch_size=5, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=5, shuffle=True)

    # train_features, train_labels = next(iter(train_dataloader))
    # print(f"Feature batch shape: {train_features.size()}")
    # print(f"Labels batch shape: {train_labels.size()}")

    # The CrossEntropy loss function in PyTorch expects the target values to be in the range
    # [0, C-1] where C is the number of classes.
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5, weight_decay=5e-4)

    epochs = 100
    for epoch in tqdm(range(1, epochs + 1)):
        print(f"Epoch {epoch}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

In [3]:
# main()

In [5]:
def read_data():
    """
    two subjects: S4b, X11b
    The experiment consists of 3 sessions for each subject. Each session consists of 4 to 9 runs
    """
    train_data = np.load('drive/MyDrive/EEGNet/processed_seed_dataset/subject1.npy')
    test_data = np.load('drive/MyDrive/EEGNet/processed_seed_dataset/subject2.npy')

    train_label = [2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 1, 2, 0] * 3
    test_label = [2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 1, 2, 0] * 3


    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

    train_data = torch.from_numpy(train_data).float()
    test_data = torch.from_numpy(test_data).float()
    train_label = torch.tensor(train_label, dtype=torch.long)
    test_label = torch.tensor(test_label, dtype=torch.long)
    val_data = test_data
    val_label = test_label

    print(train_data.shape, train_label.shape, val_data.shape, val_label.shape, test_data.shape, test_label.shape)

    return train_data, train_label, val_data, val_label, test_data, test_label


# source_data, source_label, val_data, val_label, target_data, target_label = read_data()

In [6]:
def test(x_test,y_test,model,device):

    # model.load_state_dict(torch.load(filepath))
    model.eval()
    with torch.no_grad():
        model.cuda(0)
        n = x_test.shape[0]

        # x_test = x_test.astype("float32")
        # y_test = y_test.astype("float32").reshape(y_test.shape[0],)

        x_test,y_test = x_test.to(device),y_test.to(device)
        y_pred_test, features = model(x_test)

        correct_test = (torch.max(y_pred_test,1)[1]==y_test).sum().item()
        test_accuracy = correct_test / n
        # print("testing accuracy:",correct/n)

    return test_accuracy, features


def train(source_data, source_label, val_data, val_label, target_data, target_label, epochs=500, lr=1e-3):
    max_training_accuracy = 0
    max_test_accuracy = 0
    device = torch.device("cuda:0")

    x, y = source_data, source_label

    model = EEGNet_ReLU(n_output=3)
    # print(model)
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(),lr = lr)
    # optimizer = optim.RMSprop(model.parameters(),lr = lr, momentum = 0.2)
    # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.5, weight_decay=5e-4)

    model.cuda(0)
    summary(model.cuda(),(1, 62, 37001))

    loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []
    test_accuracy_history = []

    for epoch in range(epochs):
        model.train()
        x, y = x.to(device), y.to(device)
        y_pred, source_features = model(x)

        loss = criterion(y_pred, y)
        loss_history.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        sample_cnt = y.shape[0]
        correct = (torch.max(y_pred,1)[1]==y).sum().item()
        train_accuracy = correct / sample_cnt
        train_accuracy_history.append(train_accuracy)

        val_accuracy, val_features = test(val_data, val_label, model, device)
        val_accuracy_history.append(val_accuracy)

        # test_accuracy, target_features = test(target_data, target_label, model, device)
        # test_accuracy_history.append(test_accuracy)

        print("epochs:",epoch,"loss:",loss.item(),"D_s Accuracy:",train_accuracy, "D_t accuracy", val_accuracy)

        # max_training_accuracy = max(train_accuracy, max_training_accuracy)

        # max_test_accuracy = max(test_accuracy, max_test_accuracy)



    # plt.figure(figsize=(8, 4))
    # # plt.plot(loss_history, label="Loss")
    # plt.plot(train_accuracy_history, label='Train Accuracy')
    # plt.plot(val_accuracy_history, label='Validation Accuracy')
    # plt.plot(test_accuracy_history, label='Test Accuracy')
    # plt.xlabel('Epochs')
    # plt.ylabel('Accuracy')
    # plt.legend()
    # plt.title('Accuracy Curve')
    # plt.grid(True)
    # plt.show()



source_data, source_label, val_data, val_label, target_data, target_label = read_data()
source_data = source_data[:20]
source_label = source_label[:20]
val_data = val_data[:20]
val_label = val_label[:20]
train(source_data, source_label, val_data, val_label, target_data, target_label, epochs=100, lr=1e-2)

torch.Size([45, 1, 62, 37001]) torch.Size([45]) torch.Size([45, 1, 62, 37001]) torch.Size([45]) torch.Size([45, 1, 62, 37001]) torch.Size([45])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 16, 62, 37001]             816
       BatchNorm2d-2        [-1, 16, 62, 37001]              32
              ReLU-3        [-1, 16, 62, 37001]               0
         AvgPool2d-4         [-1, 16, 62, 9250]               0
            Conv2d-5          [-1, 32, 1, 9250]          31,744
       BatchNorm2d-6          [-1, 32, 1, 9250]              64
              ReLU-7          [-1, 32, 1, 9250]               0
         AvgPool2d-8          [-1, 32, 1, 1156]               0
           Flatten-9                [-1, 36992]               0
           Linear-10                  [-1, 256]       9,470,208
             ReLU-11                  [-1, 256]               0
           Linear-12   

KeyboardInterrupt: ignored